### Предсказание будущих трат клиентов

#### Описание задачи

Одним из самых ценных источников информации о клиенте являются данные о банковских транзакциях. Необходимо предсказать будущие траты клиента, используя информацию о ранее совершенных тратах по последовательности МСС-кодов транзакций (вот [статья](https://www.banki.ru/wikibank/mcc-kod/) про МСС и [описание MCC-кодов](https://mcc-codes.ru/code)).

Метрики качества **MAP@10** и **MNDCG@10**

<div style="border:solid #64EDC1 2px; padding: 40px">

```MAP@K - Mean Average Precision at K```

$$\Large AP@K = \frac {\sum_{i=1}^{N} \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{K} \times \mathbb{1}_{Rel_i}} {|Rel|}$$

```Average Precision at K (AP@K)``` вычисляет среднюю точность для предсказаний до позиции K

- $\mathbb{1}_{r_{ij}}$ - индикаторная функция, показывающая, яаляется ли элемент в позиции j положительным для пользователя i, т.е. с ним было взаимодействие (1 - было, 0 - не было)
- $\mathbb{1}_{Rel_i}$ - индикаторная функция, показывающая, есть ли у пользователя i вообще положительные элементы (1 - да, 0 - нет)
- |Rel| - общее количество положительных элементов для всех пользователей.
- N - количество пользователей в выборке.

$$\Large MAP@K(i) = \frac{1}{N} \sum_{n=1}^N AP@K(i)_n$$

```Mean Average Precision at K (MAP@K)``` вычисляет среднюю точность для всех пользователей до позиции K

- AP@K_i - Average Precision at K для пользователя i.
- N - количество пользователей в выборке.
    
</div>

<div style="border:solid #64EDC1 2px; padding: 40px">

```MNDCG@K - Normalized Discounted Cumulative Gain at K```

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+2)}$$

```Discounted Cumulative Gain at K (DCG@K)``` вычисляет сумму значений, пропорциональных релевантности, для предсказаний до позиции K

$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$ (1 - да, 0 - нет)

$\log2(j+2)$ - логарифм по основанию 2 от j+2, используется для учета понижения значения с увеличением позиции. 2 - константа (можно уменьшить константу-шаг)

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+2)}$$

```Ideal Discounted Cumulative Gain at K (IDCG@K)``` представляет собой максимально возможное значение DCG@K для пользователя i и рекомендаций длины K

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$

$\mathbb{1}{j\leq|Reli|}$ - индикаторная функция, которая равна 1, если j $\leq$ количеству релевантных элементов $|Reli|$ для пользователя i, иначе равна 0.

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

```Normalized Discounted Cumulative Gain at K (nDCG@K)``` вычисляет относительное значение DCG@K по отношению к IDCG@K

$$\Large MnDCG@K(i) = \frac{1}{N} \sum_{n=1}^N nDCG@K(i)_n$$

```Mean Normalized Discounted Cumulative Gain at K (MnDCG@K)``` усредняет значения nDCG@K для всех пользователей
        
</div>

_____

Оценивать вероятность взаимодействия пользователя с элементом будем при помощи алгоритмв LambdaRank, который используется в задачах ранжирования. Алгоритм LambdaRank выполняет оптимизацию модели, используя функцию потерь, которая основана на попарных различиях между примерами и их предсказанными значениями. Основная идея алгоритма заключается в том, чтобы минимизировать попарные различия в позициях примеров, чтобы улучшить показатели наших метрик AP и NDCG.

<div style="border:solid #64EDC1 2px; padding: 20px">

$$\large \text{loss} = \frac{\sum \text{positive_mask} \cdot \text{DeltaNDCG}}{\sum \text{positive_mask} + \epsilon}$$

- positive_mask - маска для пар, где разница позиций положительна,
- DeltaNDCG - изменение метрики NDCG (Normalized Discounted Cumulative Gain) при замене примеров i, j местами,
- $\epsilon$ - нконстанта для стабильности деления.

функция потерь lambda_rank_loss вычисляет среднее значение DeltaNDCG для пар с положительной разницей позиций, учитывая маску positive_mask. Затем результат делится на сумму positive_mask с добавлением небольшого значения $\epsilon$ для предотвращения деления на ноль. Так и оптимизируется модель, уменьшая различия в позициях примеров
    
</div>

_____

импортируем библиотеки:

In [1]:
import csv
import random
import warnings
from tqdm import tqdm

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l1

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

метрика - MAP@K:

In [2]:
def apk(actual, predicted, k=10):
    """расчет MAP@K"""
    
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """вычисляет среднее значение MAP@K для набора actual и predicted"""
    
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

метрика - MNDCG@K:

In [3]:
def dcg_at_k(recommended_list, bought_list, k):
    """вычисляет DCG@k для заданных списков рекомендаций и фактических items"""
    
    dcg = 0.0
    for i in range(k):
        if recommended_list[i] in bought_list:
            dcg += 1.0 / np.log2(i + 2)  # числитель: считаем только совпадения, 2 - константа
    return dcg


def ndcg_at_k(recommended_list, bought_list, k):
    """расчет NDCG@k"""

    idcg = dcg_at_k(bought_list, bought_list, k)
    if idcg == 0:
        return 0.0
    dcg = dcg_at_k(recommended_list, bought_list, k)
    return dcg / idcg


def mean_ndcg_at_k(actual, predicted, k=10):
    """вычисляет среднее значение NDCG@k для набора actual и predicted"""
    
    ndcg_scores = []
    for a, p in zip(actual, predicted):
        ndcg_scores.append(ndcg_at_k(p, a, k))
    return np.mean(ndcg_scores)

загружаем файл и подготавливаем его:

In [4]:
df_train = pd.read_csv('data/df_train.csv', sep=';')
df_test = pd.read_csv('data/df_test.csv', sep=';')
sample_forecast = pd.read_csv('data/submission_baseline_2.csv')
df_train.head(3)

,Id,Data,Target
0,0,"4814,4814,6010,6011,4814,6011,6011,4814,6011,6...","4814,4814,4814,4814,5411,4814,4814,4814,4814,4814"
1,1,"6011,6011,6011,6011,6011,6011,6011,4814,4814,4...","4814,6011,4814,6011,4814,4814,6011,4814,6011,4814"
2,2,"8021,6011,6011,6010,4829,4814,6011,6011,6011,6...","6011,6011,6010,4829,4829,6010,6011,6011,4814,6011"


In [5]:
def string_processing(df):
    """функция собирает значения наблюдений в списки"""
    
    df_ = df.copy()
    df_['Data'] = df_.Data.apply(lambda s: list(map(int, s.split(','))))
    try:
        df_['Target'] = df_.Target.apply(lambda s: list(map(int, s.split(','))))
    except AttributeError:
        pass
    return df_

In [6]:
df_train = string_processing(df_train)
df_test = string_processing(df_test)
display(df_train.head(3), df_test.head(3))

,Id,Data,Target
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601..."


,Id,Data
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482..."


In [7]:
print(f'размеры:\n{df_train.shape=}\n{df_test.shape=}')
if (df_train['Id'].values == df_test['Id'].values).all():
    print("Значения в столбце 'Id' идут в одинаковом порядке.")
else:
    print("Значения в столбце 'Id' не идут в одинаковом порядке.")

размеры:
df_train.shape=(7033, 3)
df_test.shape=(7033, 2)
Значения в столбце 'Id' идут в одинаковом порядке.


проверим минимальное и максимальное количество МСС для Id в файлах:

In [8]:
test_list = list(set([len(i) for i in df_test['Data']]))
train_list = list(set([len(i) for i in df_train['Data']]))
print(f' TRAIN\nминимум МСС:{min(train_list)}\nмаксимум МСС:{max(train_list)}\n',
      f'TEST\nминимум МСС:{min(test_list)}\nмаксимум МСС:{max(test_list)}')

 TRAIN
минимум МСС:40
максимум МСС:21101
 TEST
минимум МСС:40
максимум МСС:88771


далее код класса, в котором реализована вся логика получения ранжирования:

In [17]:
class RankingForecast:
    """
    класс формирует столбец с 10-ю ранжированными МСС, основан на 
    свёрточных нейронных сетях LambdaRank
    """
    
    def __init__(self, df_train, df_test, work_check=False):
        self.df_train = pd.DataFrame([df_train])
        self.df_test = pd.DataFrame([df_test])
        self.predicted = None
        self.work_check = work_check
        self.Data_train = self.df_train['Data'].values[0]
        self.Target_train = self.df_train['Target'].values[0]
        self.Data_test = self.df_test['Data'].values[0]
        self.size_Data_train = len(self.Data_train)
        
        # на случай, если не окажется 10-ти элементов:
        self.RANDOM_STATE = 42
        self.device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
    
    def lambda_rank_loss(self, y_true, y_pred):
        """Функция потерь LambdaRank"""

        # Преобразуем y_true в тип float32
        y_true = tf.cast(y_true, dtype='float32')
        # Сортируем предсказания по убыванию и сохраняем индексы
        sorted_indices = tf.argsort(y_pred, axis=-1, direction='DESCENDING')[:, :self.size_Data_train]
        # Сортируем y_true соответствующим образом
        sorted_y_true = tf.gather(y_true, sorted_indices, axis=1)

        # Вычисляем попарные разности позиций для каждой пары примеров
        # positive_diff[i, j] - разница позиций для положительной пары i, j
        positive_diff = tf.expand_dims(sorted_y_true, axis=2) - tf.expand_dims(sorted_y_true, axis=1)
        # Вычисляем попарные разности предсказанных значений
        # pred_diff[i, j] - разница предсказанных значений для примеров i, j
        pred_diff = tf.expand_dims(y_pred, axis=2) - tf.expand_dims(y_pred, axis=1)
        # Вычисляем маску для пар, где разница позиций положительна
        positive_mask = tf.cast(positive_diff > 0, dtype='float32')
        # Вычисляем значения функции потерь LambdaRank
        # DeltaNDCG[i, j] - изменение метрики NDCG при замене примеров i, j местами
        DeltaNDCG = tf.abs(1.0 / tf.math.log(2.0 + tf.cast(positive_diff, dtype='float32'))) * tf.abs(1.0 / (1.0 + tf.exp(pred_diff)))
        # Умножаем DeltaNDCG на маску, чтобы игнорировать пары с неположительной разницей позиций
        loss = tf.reduce_sum(positive_mask * DeltaNDCG) / (tf.reduce_sum(positive_mask) + K.epsilon())

        return loss
    
    def create_model_lambda_rank(self, learning_rate=0.00001):
        """Модель LambdaRank"""

        random.seed(self.RANDOM_STATE)
        np.random.seed(self.RANDOM_STATE)
        tf.random.set_seed(self.RANDOM_STATE)

        with tf.device(self.device):
            input_layer = Input(shape=(self.size_Data_train,))
            x = Dense(32, activation='relu')(input_layer)
            x = BatchNormalization()(x)
            x = Dense(128, activation='relu')(x)
            x = BatchNormalization()(x)
            x = Dense(64, activation='sigmoid')(x)
            x = BatchNormalization()(x)
            x = Dense(32, activation='relu')(x)            
            x = Dropout(0.2)(x)
            x = Dense(16, activation='relu')(x)
            x = Dropout(0.2)(x)
            x = Flatten()(x) 
            output_layer = Dense(self.size_Data_train, activation='linear', kernel_regularizer=l1(0.01))(x)

            model = Model(inputs=input_layer, outputs=output_layer)
            model.compile(loss=self.lambda_rank_loss, optimizer=SGD(learning_rate=learning_rate))

        return model    
    
    def get_prediction(self):
        """формируем предсказание"""
        
        X_train = np.array([self.Data_train], dtype='float32')
        Y = np.array([self.Target_train + [0] * (self.size_Data_train - len(self.Target_train))])
        X_test = np.array([self.Data_test], dtype='float32')
        
        if len(X_train[0]) < len(X_test[0]):
            X_test = X_test[:, (len(X_test[0]) - len(X_train[0])):]
        if len(X_train[0]) > len(X_test[0]):
            X_test = np.array([self.Data_test + [0] * (self.size_Data_train - len(self.Data_test))])
        model = self.create_model_lambda_rank()
        with tf.device(self.device):
            model.fit(X_train, Y, epochs=50, batch_size=48, verbose=0)
            if self.work_check:
                predictions = model.predict(X_train)
            else:
                predictions = model.predict(X_test) 
            del model
        self.predicted = [x for _, x in sorted(zip(predictions[0], self.Data_test), reverse=True)][:10]
        
        return self.predicted

протестируем работу на первых десяти строчках:

In [13]:
df_train_test = df_train.loc[:10]
df_train_test

,Id,Data,Target
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601..."
3,3,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569...","[6011, 6011, 6010, 6011, 6011, 4814, 4814, 601..."
4,4,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481...","[5499, 6011, 4814, 4829, 5200, 5411, 5499, 591..."
5,5,"[5411, 4814, 6011, 6011, 5411, 7311, 5499, 481...","[6011, 5331, 5411, 5411, 6011, 6010, 6011, 601..."
6,6,"[4814, 4814, 4814, 4814, 4814, 5499, 5499, 549...","[5499, 5499, 6010, 5411, 5411, 5499, 4814, 549..."
7,7,"[6010, 5921, 6011, 6010, 6011, 6011, 6011, 601...","[6011, 6011, 6010, 6011, 6011, 6011, 6011, 601..."
8,8,"[6011, 4814, 4814, 4814, 4814, 4814, 6011, 739...","[4814, 5735, 6011, 4814, 4814, 5735, 6010, 601..."
9,9,"[5411, 5411, 5411, 5411, 5411, 5541, 6011, 541...","[5411, 5411, 5411, 4829, 5411, 5411, 5411, 541..."


In [19]:
df_train_test['Predicted'] = 0
predict_list = []
for i in range(df_train_test.shape[0]):
    train_row = df_train_test.iloc[i]
    test_row = df_train_test.iloc[i]
    prediction = RankingForecast(train_row, test_row, work_check=True).get_prediction()
    predict_list.append(prediction)
    
df_train_test['Predicted'] = predict_list

display(df_train_test)
print(f"   MAP@10 = {mapk(df_train_test['Target'], df_train_test['Predicted'])}\n",
      f"MNDCG@10 = {mean_ndcg_at_k(df_train_test['Target'], df_train_test['Predicted'])}")

1/1 [==============================] - 0s 60ms/step


,Id,Data,Target,Predicted
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481...","[5311, 6011, 6011, 6011, 4814, 5411, 4814, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481...","[4814, 4814, 6011, 6011, 4814, 4814, 4814, 601..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601...","[5814, 6010, 6011, 4829, 5211, 5331, 4814, 533..."
3,3,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569...","[6011, 6011, 6010, 6011, 6011, 4814, 4814, 601...","[4814, 5411, 5411, 6011, 4814, 5411, 5411, 541..."
4,4,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481...","[5499, 6011, 4814, 4829, 5200, 5411, 5499, 591...","[6011, 4829, 6011, 6011, 5411, 4814, 5200, 549..."
5,5,"[5411, 4814, 6011, 6011, 5411, 7311, 5499, 481...","[6011, 5331, 5411, 5411, 6011, 6010, 6011, 601...","[6011, 6011, 5411, 6011, 5912, 6011, 5411, 599..."
6,6,"[4814, 4814, 4814, 4814, 4814, 5499, 5499, 549...","[5499, 5499, 6010, 5411, 5411, 5499, 4814, 549...","[5411, 5411, 4814, 4814, 5499, 5499, 5499, 549..."
7,7,"[6010, 5921, 6011, 6010, 6011, 6011, 6011, 601...","[6011, 6011, 6010, 6011, 6011, 6011, 6011, 601...","[6011, 6011, 6011, 6011, 6010, 5921, 6011, 601..."
8,8,"[6011, 4814, 4814, 4814, 4814, 4814, 6011, 739...","[4814, 5735, 6011, 4814, 4814, 5735, 6010, 601...","[5735, 6011, 5735, 4814, 4814, 4814, 4814, 739..."
9,9,"[5411, 5411, 5411, 5411, 5411, 5541, 6011, 541...","[5411, 5411, 5411, 4829, 5411, 5411, 5411, 541...","[5411, 5541, 5411, 5411, 5411, 6011, 5411, 481..."


   MAP@10 = 0.20002164502164502
 MNDCG@10 = 0.7994613318619105


_______

_______

формируем столбцы с предсказанным ранжированием на тренировочной выборке, предварительно порезав её на отдельные файлы, иначе памяти не хватает:

In [20]:
def csv_splitter(file_name: str, chunk_size: int, header=None, get_names=False):
    """
    функция делит один csv-файл на несколько меньших по chunk_size - количеству заданных строк
    можно добавить заголовки и вернуть список с названием созданных файлов
    """
    
    name_files_list = []
    with open(f'{file_name}.csv', 'r') as f:
        reader = csv.reader(f)
        header = next(reader)          # заголовки
        rows = [row for row in reader]

    num_chunks = len(rows) // chunk_size + 1
    count = 0
    for i in range(num_chunks):
        start_index = i * chunk_size
        end_index = start_index + chunk_size
        chunk = rows[start_index:end_index]
        if i == num_chunks - 1 and len(rows) % chunk_size != 0:
            last_chunk = rows[i * chunk_size:]
            output_file = f'{file_name}_small_{i}.csv'
            with open(output_file, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(header)
                writer.writerows(last_chunk)
        else:
            output_file = f'{file_name}_small_{i}.csv'
            with open(output_file, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(header)
                writer.writerows(chunk)
        count = i + 1
        name_files_list.append(output_file)
    print(f'успешный успех! Файл "{file_name}" разделён на {count} частей')
    if get_names is not False:
        return name_files_list

In [21]:
train_files_list = csv_splitter('data/df_train', 600, get_names=True)
test_files_list = csv_splitter('data/df_test', 600, get_names=True)
print(f'TRAIN names:\n{train_files_list}\n\nTEST names:\n{test_files_list}')

успешный успех! Файл "data/df_train" разделён на 12 частей
успешный успех! Файл "data/df_test" разделён на 12 частей
TRAIN names:
['data/df_train_small_0.csv', 'data/df_train_small_1.csv', 'data/df_train_small_2.csv', 'data/df_train_small_3.csv', 'data/df_train_small_4.csv', 'data/df_train_small_5.csv', 'data/df_train_small_6.csv', 'data/df_train_small_7.csv', 'data/df_train_small_8.csv', 'data/df_train_small_9.csv', 'data/df_train_small_10.csv', 'data/df_train_small_11.csv']

TEST names:
['data/df_test_small_0.csv', 'data/df_test_small_1.csv', 'data/df_test_small_2.csv', 'data/df_test_small_3.csv', 'data/df_test_small_4.csv', 'data/df_test_small_5.csv', 'data/df_test_small_6.csv', 'data/df_test_small_7.csv', 'data/df_test_small_8.csv', 'data/df_test_small_9.csv', 'data/df_test_small_10.csv', 'data/df_test_small_11.csv']


_____

In [18]:
# train_files_list = ['data/df_train_small_0.csv', 'data/df_train_small_1.csv',
#                     'data/df_train_small_2.csv', 'data/df_train_small_3.csv',
#                     'data/df_train_small_4.csv', 'data/df_train_small_5.csv',
#                     'data/df_train_small_6.csv', 'data/df_train_small_7.csv',
#                     'data/df_train_small_8.csv', 'data/df_train_small_9.csv',
#                     'data/df_train_small_10.csv', 'data/df_train_small_11.csv']
# test_files_list = ['data/df_test_small_0.csv', 'data/df_test_small_1.csv',
#                    'data/df_test_small_2.csv', 'data/df_test_small_3.csv',
#                    'data/df_test_small_4.csv', 'data/df_test_small_5.csv',
#                    'data/df_test_small_6.csv', 'data/df_test_small_7.csv',
#                    'data/df_test_small_8.csv', 'data/df_test_small_9.csv',
#                    'data/df_test_small_10.csv','data/df_test_small_11.csv']

### погнали предсказывать:

In [22]:
def get_predict(train_name, test_name):
    """функция формирует признак с предсказанием и сохраняет файл под новым названием"""
    
    data_train = string_processing(pd.read_csv(train_name, sep=';'))
    data_test = string_processing(pd.read_csv(test_name, sep=';'))
    
    data_test['Predicted'] = 0
    predict_list = []
    for i in range(data_train.shape[0]):
        train_row = data_train.iloc[i]
        test_row = data_test.iloc[i]
        prediction = RankingForecast(train_row, test_row).get_prediction()
        predict_list.append(prediction)
    data_test['Predicted'] = predict_list
    new_name = test_name+'_LambdaRankPredicted'
    data_test.to_csv(new_name, index=False)
    return 'успех!'

In [20]:
# %%time
# get_predict(train_files_list[11], test_files_list[11])

1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 47ms/step
CPU times: total: 12min 24s
Wall time: 7min 54s


'успех!'

собираем файлы вместе:

In [21]:
data_response_file_name = [f'data/df_test_small_{i}.csv_LambdaRankPredicted' for i in range(12)]
data_response_file_name

['data/df_test_small_0.csv_LambdaRankPredicted',
 'data/df_test_small_1.csv_LambdaRankPredicted',
 'data/df_test_small_2.csv_LambdaRankPredicted',
 'data/df_test_small_3.csv_LambdaRankPredicted',
 'data/df_test_small_4.csv_LambdaRankPredicted',
 'data/df_test_small_5.csv_LambdaRankPredicted',
 'data/df_test_small_6.csv_LambdaRankPredicted',
 'data/df_test_small_7.csv_LambdaRankPredicted',
 'data/df_test_small_8.csv_LambdaRankPredicted',
 'data/df_test_small_9.csv_LambdaRankPredicted',
 'data/df_test_small_10.csv_LambdaRankPredicted',
 'data/df_test_small_11.csv_LambdaRankPredicted']

In [22]:
data_response = \
    pd.concat([pd.read_csv(name, sep=',') for name in data_response_file_name], ignore_index=True)
print(f'{data_response.shape=}')
display(data_response.head(3))

data_response.shape=(7033, 3)


,Id,Data,Predicted
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481...","[6011, 6011, 4814, 6011, 4814, 6011, 5541, 482..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601...","[6010, 6011, 5411, 6010, 6011, 6011, 5691, 481..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482...","[5999, 5814, 6011, 5499, 7832, 5814, 4814, 601..."


In [23]:
data_response.to_csv('data/predict/data_and_LambdaRank.csv', index=False)
predicted_files = data_response[['Id', 'Predicted']]
predicted_files.head(5)

,Id,Predicted
0,0,"[6011, 6011, 4814, 6011, 4814, 6011, 5541, 482..."
1,1,"[6010, 6011, 5411, 6010, 6011, 6011, 5691, 481..."
2,2,"[5999, 5814, 6011, 5499, 7832, 5814, 4814, 601..."
3,3,"[4814, 5411, 4829, 4814, 6011, 4814, 4814, 481..."
4,4,"[6011, 4814, 6011, 4814, 4814, 4814, 6011, 481..."


In [25]:
predicted_files['Predicted'] = predicted_files.Predicted.astype(str).str.replace(',', '')
predicted_files

,Id,Predicted
0,0,[6011 6011 4814 6011 4814 6011 5541 4829 4814 ...
1,1,[6010 6011 5411 6010 6011 6011 5691 4814 5411 ...
2,2,[5999 5814 6011 5499 7832 5814 4814 6011 4814 ...
3,3,[4814 5411 4829 4814 6011 4814 4814 4814 5977 ...
4,4,[6011 4814 6011 4814 4814 4814 6011 4814 6011 ...
...,...,...
7028,7028,[5411 6011 5499 5499 6011 5411 6011 5411 8099 ...
7029,7029,[5411 5411 5411 5331 5411 5411 5411 5411 5411 ...
7030,7030,[4814 5331 4814 4814 4814 5499 5541 6011 5499 ...
7031,7031,[6011 6011 4814 6011 4814 6011 6011 6011 6011 ...


In [26]:
predicted_files.to_csv('data/predict/8_2_submission_LambdaRank.csv', index=False)

модель показывает хорошее значение MNDCG@10 на тестовой выборке, но MAP@10 ниже baseline_модели. Это может говорить о том, что модель предсказывает достаточно релевантные МСС-коды, но не всегда уверена в их порядке, над порядком ещё стоит поработать

_____